# 初期設定

## 1. Azure Machine Learning service Python SDK バージョン確認

In [1]:
import azureml.core

print("This notebook was created using version 1.0.23 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.23 of the Azure ML SDK
You are currently using version 1.0.23 of the Azure ML SDK


## 2. Azure Machine Learning service Workspaceの認証

接続情報を記録する config.json を作成
- subscription_id : サブスクリプションのID
- resource_group : リソースグループ名
- workspace_name : Azure Machine Learning service ワークスペース名称

In [2]:
from azureml.core import Workspace

try:
    ws = Workspace(
        subscription_id = "9c0f91b8-eb2f-484c-979c-15848c098a6b", 
        resource_group = "dllab", 
        workspace_name = "azureml"
    )

    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible. Change your parameters or create a new workspace below")

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace configuration succeeded.


In [3]:
ws.from_config()
print(ws.subscription_id, ws.resource_group, ws.name)

Found the config file in: /Users/konabuta/Project/Manufacturing-ML/.azureml/config.json
9c0f91b8-eb2f-484c-979c-15848c098a6b dllab azureml


## Python SDK アップデート

In [4]:
#!pip install --upgrade azureml-sdk[notebooks,automl,explain,contrib] azureml-dataprep

## クラスター構築

注意： 無償のサブスクリプションを使用されている人は、STANDARD_D2_V2 を使用してください

## GPUの場合

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# クラスター名の指定
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_NC6", ## GPU/CPUインスタンスの種類 
                                                       min_nodes=1, # 最小ノード数
                                                       max_nodes=1, # 最大ノード数
                                                       vm_priority='lowpriority') ## lowpriority = 低優先 | dedicated = 専用

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2019-04-18T05:19:04.834000+00:00', 'errors': None, 'creationTime': '2019-02-22T01:18:20.614322+00:00', 'modifiedTime': '2019-04-01T00:06:16.538379+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'LowPriority', 'vmSize': 'STANDARD_NC6'}


## CPUの場合

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# クラスター名の指定
cluster_name = "cpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2", ## GPU/CPUインスタンスの種類 
                                                       min_nodes=1, # 最小ノード数
                                                       max_nodes=1, # 最大ノード数
                                                       vm_priority='lowpriority') ## lowpriority = 低優先 | dedicated = 専用

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current cluster. 
print(compute_target.status.serialize())

Found existing compute target.
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 1, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-04-18T05:33:25.317000+00:00', 'errors': None, 'creationTime': '2019-02-22T01:22:47.309361+00:00', 'modifiedTime': '2019-04-09T11:16:51.118659+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT600S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}
